In [ ]:
#Importing Necessary Libraries
import numpy as np
import pandas as pd
import os
import math
import re
import time
import json
import gc
from collections import Counter
from itertools import tee, islice
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm_notebook
import nltk
import pickle
import gzip
import pickletools
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
path_to_json = 'pdf_json/'
drive_directory = "/content/drive/MyDrive/"

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
%%time
!unzip /content/drive/MyDrive/pdf_json.zip > /dev/null

CPU times: user 438 ms, sys: 66.1 ms, total: 504 ms
Wall time: 59.7 s


In [ ]:
!ls /content/pdf_json > doc.txt

##$\color{yellow}{\text{Data Extraction}}$

In [ ]:
#Taking only first 50000 files for corpus
with open('doc.txt') as f:
    lines = f.readlines()
lines = lines[:50000]   

In [ ]:

def extract_text(text):
    #Extracting the body text from each document
    content = [text['body_text'][j]['text'] for j in range(len(text['body_text']))]
    full_text = content
    return full_text

final_text=[]
for i in tqdm_notebook(range(len(lines))):
    path = '/content/pdf_json/'+lines[i].strip("\n")
    text = json.load(open(path))
    final_text.append(extract_text(text))

  0%|          | 0/50000 [00:00<?, ?it/s]

In [ ]:
#Joining the sentences into string  format
final_str_text = [' '.join(final_text[i]) for i in tqdm_notebook(range(len(final_text)))]

  0%|          | 0/50000 [00:00<?, ?it/s]

In [ ]:
#Saving the initial corpus as txt file
with open(drive_directory+'inital_corpus.txt', 'w') as f:
    for words in tqdm_notebook(range(len(final_str_text))):
        if words != '':
            #print(final_pad_str_text[words])
            f.write("%s\n" % final_str_text[words])

  0%|          | 0/50000 [00:00<?, ?it/s]

In [ ]:
#Padding each sentence with start and end tag, here start tag is 'senstart' and end tag is 'senend'
def padding(n):
    l=[]
    for i in tqdm_notebook(range(len(final_str_text))):
        ank = final_str_text[i]
        pad_sen = ["senstart "*(n-1) + j.lower() +  " senend"*(n-1) for j in sent_tokenize(ank)]
        l.append(pad_sen)
    return l

## $\color{yellow}{\text{Bigram Corpus Creation}}$

In [ ]:
bigram_text = padding(2)

  0%|          | 0/50000 [00:00<?, ?it/s]

In [ ]:
final_bigram_pad_str_text = [' '.join(bigram_text[i]) for i in tqdm_notebook(range(len(bigram_text)))]

  0%|          | 0/50000 [00:00<?, ?it/s]

##$\color{yellow}{\text{Preprocessing The Bigram Data}}$

In [ ]:
def preprocessing(data):
    # data = data.lower()
    # STOPWORDS = set(stopwords.words('english'))
    # text_1 = " ".join([word for word in data.split() if word not in STOPWORDS])
    
    #Removing Email 
    text_2 = re.sub(r"\S*@\S*\s?",'', data)
    
    #Removing Punctuation
    text_3 = re.sub(r'[^\w\s]', '', text_2)

    #Removing URLs
    text_4 = re.sub(r'^https?:\/\/.*[\r\n]*', '',text_3)

    #Removing dates
    text_5 = re.sub(r'^(?:(?:[0-9]{2}[:\/,]){2}[0-9]{2,4}|am|pm)$', '', text_4)

    #Removing non-alphanumeric characters
    text_6 = re.sub(r'\W+', ' ', text_5)

    #Removing numbers
    text_7 = re.sub(r'[0-9]+', '', text_6)

    #Lemmatization of the words
    # wordnet_lemmatizer = WordNetLemmatizer()
    # text_7 = " ".join([wordnet_lemmatizer.lemmatize(word) for word in text_6.split(" ")])

    return text_7

In [ ]:
#Saving the bigram prepprocessed text
with open(drive_directory+'final_bigram_corpus.txt', 'w') as f:
    for words in tqdm_notebook(range(len(final_bigram_pad_str_text))):
        if words != '':
            #print(final_pad_str_text[words])
            f.write("%s\n" % preprocessing(final_bigram_pad_str_text[words]))

  0%|          | 0/50000 [00:00<?, ?it/s]

In [ ]:
with open('/content/drive/MyDrive/final_bigram_corpus.txt', 'r') as f:
    bigram_preprocessed_text = f.readlines()

##$\color{yellow}{\text{Unigram Model}}$

In [ ]:
#Function for building n-gram module 
def ngrams(lst, n):
  tlst = lst
  while True:
    a, b = tee(tlst)
    l = tuple(islice(a, n))
    if len(l) == n:
      yield l
      next(b)
      tlst = b
    else:
      break

In [ ]:
#Creating Unigram model and saving it
%%time
coun = Counter()
for i in tqdm_notebook(range(len(bigram_preprocessed_text))):
    w = bigram_preprocessed_text[i].split()
    coun.update(ngrams(w,1))
unigrams = dict(coun)
with open(drive_directory+'unigrams', 'wb') as f:
    pickle.dump(unigrams,f)
    f.close() 

  0%|          | 0/50000 [00:00<?, ?it/s]

CPU times: user 2min 46s, sys: 887 ms, total: 2min 47s
Wall time: 2min 53s


In [ ]:
with open("/content/drive/MyDrive/unigrams","rb") as f:
    uni = pickle.load(f)
#unique_words= uni.keys()
#print("Number of unique words in the corpus is =",len(unique_words)-2)
#del(unique_words)

In [ ]:
#Vocabulary Count
print("Number of unique words in the corpus is =",len(uni.keys())-2)

Number of unique words in the corpus is = 1353665


##$\color{yellow}{\text{Bigram Model}}$

In [ ]:
#Creating Bigram model and saivng it
coun1 = Counter()
for i in tqdm_notebook(range(len(bigram_preprocessed_text))):
    w = bigram_preprocessed_text[i].split()
    coun1.update(ngrams(w,2))
bigrams = dict(coun1)
with open(drive_directory+'bigrams', 'wb') as f:
    pickle.dump(bigrams,f)
    f.close()  

  0%|          | 0/50000 [00:00<?, ?it/s]

In [ ]:
with open('/content/drive/MyDrive/bigrams','rb') as f:
    bigram = pickle.load(f)


##$\color{yellow}{\text{Next Word Prediction Using Bigram}}$

In [ ]:
#This function is is checking if a key is present in the dictonary or not, if not then it will return 0
def value(d,key):
    c = d.get(key)
    if c:
        return d[key]
    else:
        return 0    

In [ ]:
#Calculating probability of next word given first word using bigram
def prob_next_word_bigram(first_word):
  prob_next={}
  for keys in bigram.keys():
    if keys[0]== first_word:
      #Applying Laplacian +1 smoothing
      prob =  round((value(bigram,keys)+1)/(value(uni,tuple((first_word,)))+len(uni.keys())),5)
      prob_next[keys]=prob
  return dict(sorted(prob_next.items(), key=lambda x:x[1], reverse=True))

In [ ]:
"all houses were ______ ventilated"
dict(islice(prob_next_word_bigram("were").items(), 10))

{('were', 'not'): 0.01237,
 ('were', 'used'): 0.01203,
 ('were', 'also'): 0.00921,
 ('were', 'performed'): 0.00891,
 ('were', 'found'): 0.00783,
 ('were', 'collected'): 0.00751,
 ('were', 'obtained'): 0.00634,
 ('were', 'observed'): 0.00588,
 ('were', 'identified'): 0.00578,
 ('were', 'detected'): 0.00507}

In [ ]:
'''it aims to develop an integrated ______ to reach mmps exposed to malaria with prevention diagnosis and treatment ______ by 
involving non-health ______ stakeholders from provincial to community level '''
dict(islice(prob_next_word_bigram("integrated").items(), 10))

{('integrated', 'into'): 0.00115,
 ('integrated', 'with'): 0.0006,
 ('integrated', 'in'): 0.00033,
 ('integrated', 'and'): 0.00028,
 ('integrated', 'dna'): 0.0002,
 ('integrated', 'approach'): 0.0002,
 ('integrated', 'to'): 0.00013,
 ('integrated', 'the'): 0.00012,
 ('integrated', 'moving'): 0.0001,
 ('integrated', 'care'): 0.0001}

In [ ]:
'''it aims to develop an integrated ______ to reach mmps exposed to malaria with prevention diagnosis and treatment ______ by 
involving non-health ______ stakeholders from provincial to community level '''
dict(islice(prob_next_word_bigram("treatment").items(), 10))

{('treatment', 'of'): 0.02275,
 ('treatment', 'senend'): 0.01321,
 ('treatment', 'with'): 0.00895,
 ('treatment', 'and'): 0.00635,
 ('treatment', 'for'): 0.00511,
 ('treatment', 'in'): 0.00314,
 ('treatment', 'is'): 0.00301,
 ('treatment', 'was'): 0.00248,
 ('treatment', 'options'): 0.00132,
 ('treatment', 'group'): 0.00132}

In [ ]:
'''it aims to develop an integrated ______ to reach mmps exposed to malaria with prevention diagnosis and treatment ______ by 
involving non-health ______ stakeholders from provincial to community level '''
dict(islice(prob_next_word_bigram("health").items(), 10))

{('health', 'care'): 0.01509,
 ('health', 'and'): 0.01086,
 ('health', 'senend'): 0.00624,
 ('health', 'organization'): 0.00598,
 ('health', 'system'): 0.00319,
 ('health', 'services'): 0.00291,
 ('health', 'of'): 0.00256,
 ('health', 'systems'): 0.00253,
 ('health', 'emergency'): 0.00208,
 ('health', 'professionals'): 0.00199}

In [ ]:
'''this is because engineers do not work in ______ but rather as a team'''
dict(islice(prob_next_word_bigram("in").items(), 10))

{('in', 'the'): 0.17573,
 ('in', 'a'): 0.0331,
 ('in', 'this'): 0.02367,
 ('in', 'addition'): 0.01334,
 ('in', 'patients'): 0.01082,
 ('in', 'our'): 0.00826,
 ('in', 'order'): 0.00775,
 ('in', 'which'): 0.00748,
 ('in', 'vitro'): 0.00632,
 ('in', 'an'): 0.00607}

##$\color{yellow}{\text{Perplexity Score uisng Bigram}}$

In [ ]:
def pred_score_bigram(word_1,word_2):
    w = tuple((word_1,word_2))
    prob_score = (bigram[w]+1)/(value(uni,uni[tuple((word_1,))])+len(uni.keys()))
    return prob_score

In [ ]:
def bigram_perplixity_score(text):
    l1=[]
    l = text.split(" ")
    for i in range(len(l)-1):
        l1.append(pred_score_bigram(l[i],l[i+1]))
    return np.power(1/np.prod(l1),(1/(len(l)-1)))



In [ ]:
t1 = "senstart "+"half a century ago hypertension was not treatable" +" senend"
bigram_perplixity_score(preprocessing(t1))

6207.0703268813195

In [ ]:
t2 = "senstart "+"sarahs tv is broadcasting an advert for private healthcare"+" senend"
bigram_perplixity_score(preprocessing(t2))

72527.4678466028

In [ ]:
t3 = "senstart "+"it appears that the overall code stroke volume has decreased since the covid- pandemic."+" senend"
bigram_perplixity_score(preprocessing(t3))

549.5096348812219

##$\color{yellow}{\text{Preprocessing The Trigarm Data}}$

In [ ]:
trigram_text = padding(3)

  0%|          | 0/50000 [00:00<?, ?it/s]

In [ ]:
final_trigram_pad_str_text = [' '.join(trigram_text[i]) for i in tqdm_notebook(range(len(trigram_text)))]

  0%|          | 0/50000 [00:00<?, ?it/s]

In [ ]:
#Creating the trigram preprocessed text and saving it
with open(drive_directory+'final_trigram_corpus.txt', 'w') as f:
    for words in tqdm_notebook(range(len(final_trigram_pad_str_text))):
        if words != '':
            #print(final_pad_str_text[words])
            f.write("%s\n" % preprocessing(final_trigram_pad_str_text[words]))

  0%|          | 0/50000 [00:00<?, ?it/s]

In [ ]:
with open('/content/drive/MyDrive/final_trigram_corpus.txt', 'r') as f:
    trigram_preprocessed_text = f.readlines()

##$\color{yellow}{\text{Trigram Model}}$

In [ ]:
#creating trigram model
coun2 = Counter()
for i in range(len(trigram_preprocessed_text)):
    w = trigram_preprocessed_text[i].split()
    coun2.update(ngrams(w,3))
trigrams = dict(coun2)
del(coun2)
del(trigram_preprocessed_text)
gc.collect()
# with gzip.open(drive_directory+'trigrams', 'wb') as f:
#     pickled = pickle.dumps(trigrams)
#     optimized_pickle = pickletools.optimize(pickled)
#     f.write(optimized_pickle)  

0

In [ ]:
with open(drive_directory+'trigrams', 'wb') as f:
    pickle.dump(trigrams,f)
    f.close()  

##$\color{yellow}{\text{Next Word Prediction Using Trigram}}$

In [ ]:
#Calculating probability of next word given first word using trigram
def prob_next_word_trigram(first_word,second_word):
    if tuple((first_word,second_word)) == ('senstart','senstart'):
        prob_next={}
        k = tuple((first_word,second_word))
        for keys in trigrams.keys():
            if tuple((keys[0],keys[1])) == k:
                prob = (value(trigrams,keys)+1)/(uni[('senstart',)]+(len(bigram.keys())))
                prob_next[keys] = prob
        return dict(sorted(prob_next.items(), key=lambda x:x[1], reverse=True))
    else:
        prob_next={}
        k = tuple((first_word,second_word))
        for keys in trigrams.keys():
            if tuple((keys[0],keys[1])) == k:
                prob = (value(trigrams,keys)+1)/(value(bigram,k)+(len(bigram.keys())))
                prob_next[keys] = prob
        return dict(sorted(prob_next.items(), key=lambda x:x[1], reverse=True))




In [ ]:
"all houses were ______ ventilated"
dict(islice(prob_next_word_trigram("houses","were").items(), 10))

{('houses', 'were', 'investigated'): 1.0158576393357957e-07,
 ('houses', 'were', 'malaria'): 1.0158576393357957e-07,
 ('houses', 'were', 'made'): 1.0158576393357957e-07,
 ('houses', 'were', 'contacted'): 1.0158576393357957e-07,
 ('houses', 'were', 'tested'): 1.0158576393357957e-07}

In [ ]:
'''it aims to develop an integrated ______ to reach mmps exposed to malaria with prevention diagnosis and treatment ______ by 
involving non-health ______ stakeholders from provincial to community level '''
dict(islice(prob_next_word_trigram("an","integrated").items(), 10))

{('an', 'integrated', 'approach'): 1.6760340889081232e-06,
 ('an', 'integrated', 'and'): 1.3713006181975554e-06,
 ('an', 'integrated', 'system'): 6.094669414211358e-07,
 ('an', 'integrated', 'health'): 3.555223824956625e-07,
 ('an', 'integrated', 'framework'): 3.047334707105679e-07,
 ('an', 'integrated', 'view'): 3.047334707105679e-07,
 ('an', 'integrated', 'model'): 3.047334707105679e-07,
 ('an', 'integrated', 'platform'): 3.047334707105679e-07,
 ('an', 'integrated', 'assessment'): 3.047334707105679e-07,
 ('an', 'integrated', 'program'): 2.539445589254732e-07}

In [ ]:
'''it aims to develop an integrated ______ to reach mmps exposed to malaria with prevention diagnosis and treatment ______ by 
involving non-health ______ stakeholders from provincial to community level '''
dict(islice(prob_next_word_trigram("and","treatment").items(), 10))

{('and', 'treatment', 'of'): 3.35039836744225e-05,
 ('and', 'treatment', 'senend'): 1.1777157897675789e-05,
 ('and', 'treatment', 'with'): 4.467197823256334e-06,
 ('and', 'treatment', 'for'): 2.63970780465147e-06,
 ('and', 'treatment', 'is'): 2.233598911628167e-06,
 ('and', 'treatment', 'and'): 2.081308076744428e-06,
 ('and', 'treatment', 'in'): 1.9797808534886025e-06,
 ('and', 'treatment', 'strategies'): 1.9797808534886025e-06,
 ('and', 'treatment', 'options'): 1.9797808534886025e-06,
 ('and', 'treatment', 'are'): 1.4721447372094736e-06}

In [ ]:
'''it aims to develop an integrated ______ to reach mmps exposed to malaria with prevention diagnosis and treatment ______ by 
involving non-health ______ stakeholders from provincial to community level '''
dict(islice(prob_next_word_trigram("non","health").items(), 10))

{}

In [ ]:
'''this is because engineers do not work in ______ but rather as a team'''
dict(islice(prob_next_word_trigram("work","in").items(), 10))

{('work', 'in', 'the'): 1.0765669660707733e-05,
 ('work', 'in', 'this'): 3.5547022464601006e-06,
 ('work', 'in', 'a'): 3.300794943141522e-06,
 ('work', 'in', 'ensuring'): 1.1171921346017459e-06,
 ('work', 'in', 'progress'): 8.125033706194516e-07,
 ('work', 'in', 'concert'): 7.617219099557359e-07,
 ('work', 'in', 'our'): 6.093775279645887e-07,
 ('work', 'in', 'order'): 6.093775279645887e-07,
 ('work', 'in', 'an'): 5.585960673008729e-07,
 ('work', 'in', 'which'): 5.585960673008729e-07}

##$\color{yellow}{\text{Perplexity Score Using Trigram}}$

In [ ]:
def pred_score_trigram(word_1,word_2,word_3):
    if (word_1,word_2)==("senstart","senstart"):
        w = tuple((word_1,word_2,word_3))
        prob_score = (value(trigrams,w)+1)/(uni[('senstart',)]+len(bigram.keys()))
        return prob_score
    else:
        w = tuple((word_1,word_2,word_3))
        prob_score = (value(trigrams,w)+1)/(value(bigram,tuple((word_1,word_2)))+len(bigram.keys()))
        return prob_score

In [ ]:
def trigram_perplixity_score(text):
    l1=[]
    l = text.split(" ")
    for i in range(len(l)-2):
        l1.append(pred_score_trigram(l[i],l[i+1],l[i+2]))
    return np.power(1/np.prod(l1),(1/(len(l)-2)))

In [ ]:
t1 = "senstart "+"senstart "+"half a century ago hypertension was not treatable" +" senend"+" senend"
trigram_perplixity_score(preprocessing(t1))

3854532.6789549487

In [ ]:
t2 = "senstart "+"senstart "+"sarahs tv is broadcasting an advert for private healthcare"+" senend"+" senend"
trigram_perplixity_score(preprocessing(t2))

12351309.734421063

In [ ]:
t3 = "senstart "+"senstart "+"it appears that the overall code stroke volume has decreased since the covid- pandemic"+" senend"+" senend"
trigram_perplixity_score(preprocessing(t3))

283451.89810671814

In [ ]:
"hello dskcndskj dlsnc and dmedeiod ewjnewnf and emdefw dkejfbkw and ejfbewfn ndewjnf".split(" and ")

['hello dskcndskj dlsnc',
 'dmedeiod ewjnewnf',
 'emdefw dkejfbkw',
 'ejfbewfn ndewjnf']